In [2]:
!pip install transformers

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/opt/homebrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [6]:
import torch 
from transformers import BertTokenizer, BertModel 
import logging 
import matplotlib.pyplot as plt

%matplotlib inline 

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
"""
After importing the pretrained model. We will need to alter the input of our data in order to put it into the bert model
"""

text = "After stealing money from the bank vault, the bank robber was seen " \
       "fishing on the Mississippi river bank."

marked_text = "[CLS]" + text + "[SEP]"

# this is where it tokenize text into tokens 
tokenize_text = tokenizer.tokenize(marked_text)

# this is converting tokens into their respective ids
indexed_token = tokenizer.convert_tokens_to_ids(tokenize_text)

for tup in zip(tokenize_text, indexed_token):
    print('{:<12} {:>6,}'.format(tup[0], tup[1]))

[CLS]           101
after         2,044
stealing     11,065
money         2,769
from          2,013
the           1,996
bank          2,924
vault        11,632
,             1,010
the           1,996
bank          2,924
robber       27,307
was           2,001
seen          2,464
fishing       5,645
on            2,006
the           1,996
mississippi   5,900
river         2,314
bank          2,924
.             1,012
[SEP]           102


In [9]:
"""
This is where you need to let BERT know that the token is one sentence or two sentence
"""
segments_id = [1] * len(tokenize_text)

segments_id

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [11]:
"""
Converting into a tensor instead of a list
"""
token_tensor = torch.tensor([indexed_token])
segments_tensor = torch.tensor([segments_id])

In [13]:
"""
Loading the pretrained model 
"""

model = BertModel.from_pretrained("bert-base-uncased", output_hidden_states = True)

model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [21]:
"""
Running our text through BERT
"""
with torch.no_grad():

    outputs = model(token_tensor, segments_tensor)

    hidden_states = outputs[2]
    

In [23]:
print ("Number of layers:", len(hidden_states), "  (initial embeddings + 12 BERT layers)")
layer_i = 0

print ("Number of batches:", len(hidden_states[layer_i]))
batch_i = 0

print ("Number of tokens:", len(hidden_states[layer_i][batch_i]))
token_i = 0

print ("Number of hidden units:", len(hidden_states[layer_i][batch_i][token_i]))

Number of layers: 13   (initial embeddings + 12 BERT layers)
Number of batches: 1
Number of tokens: 22
Number of hidden units: 768


In [36]:
hidden_states[0].size()

torch.Size([1, 22, 768])

In [40]:
token_embeddings = torch.stack(hidden_states, dim=0)
token_embeddings.size()

torch.Size([13, 1, 22, 768])

In [43]:
token_embeddings = torch.squeeze(token_embeddings, dim=1)
token_embeddings.size()

torch.Size([13, 22, 768])

In [44]:
token_embeddings = token_embeddings.permute(1,0,2)
token_embeddings.size()

torch.Size([22, 13, 768])

Word Representation 

In [47]:
"""
For each of our token we have 13 seperate vectors each of length 768
- we need a combination of layers to provides the best representation
"""

# Concatenation of the last four layers 

tokens_vecs_cat = []

for token in token_embeddings:

    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)

    tokens_vecs_cat.append(cat_vec)

print ('Shape is: %d x %d' % (len(tokens_vecs_cat), len(tokens_vecs_cat[0])))

Shape is: 22 x 3072


In [52]:
# alternate method is summing together the last four layers

"""
Each word is a representation with 768 vector
"""
token_vecs_sum = []

for token in token_embeddings:

    sum_vec = torch.sum(token[-4:], dim= 0)

    token_vecs_sum.append(sum_vec)

print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))


Shape is: 22 x 768


Sentence Representation

In [53]:
"""
Getting a single vector for our entire sentence. The simple approach is average the second to last hidden layer
"""
token_vecs = hidden_states[-2][0]

sentence_embedding = torch.mean(token_vecs, dim=0)

In [54]:
sentence_embedding.size()

torch.Size([768])

Computing the cosine similarity 


In [57]:
from scipy.spatial.distance import cosine 

# Comparing the word 'bank' in the sentence 

_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[6])

# close to 1 mean it is very similar.
_bank

0.938639223575592